# CNN model notebook

In this notebook we are going to define and train the CNN model used for the classification task.

## Setting up the stage

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os
import json
from datetime import datetime

#Import the custom script
import ValidationCreation
import DatasplitJsonCreate

Define useful variables

In [ ]:
cwd = os.getcwd()
datasetDir = os.path.join(cwd,'Classification_Dataset')

#Classes for our problem

classes = ['owl',
'galaxy',
'lightning',
'wine-bottle',
't-shirt',
'waterfall',
'sword',
'school-bus',
'calculator',
'sheet-music',
'airplanes',
'lightbulb',
'skyscraper',
'mountain-bike',
'fireworks',
'computer-monitor',
'bear',
'grand-piano',
'kangaroo',
'laptop']

seed = 1234
tf.random.set_seed(seed)

Functions in the external scripts we created in order to split the dataset

In [ ]:
ValidationCreation.create_validation(ratio = 0.2)
DatasplitJsonCreate.DatasetSplitJSON()

Creation of the various directory paths

In [ ]:
trainingDir = os.path.join(datasetDir,'training')
tb_dir = os.path.join(cwd,'tensorboard')
sv_dir = os.path.join(cwd,'saved_model')
csv_dir = os.path.join(cwd,'csv_savings')
vd_dir = os.path.join(datasetDir,'validation')

if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
else:
    for(root,dirs,files) in os.walk(tb_dir,topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root,name))

if not os.path.exists(sv_dir):
    os.makedirs(sv_dir)

if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

Building the generator for both training and validation dataset

In [ ]:
dg = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

augment_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                                                rotation_range=20,
                                                width_shift_range=20,
                                                height_shift_range=20,
                                                zoom_range=0.2,
                                                horizontal_flip=True,
                                                fill_mode='constant',
                                                cval = 0,
                                                shear_range = 0.2,
                                                rescale=1./255
                                                )

bs = 20 #training Batch Size
bsv = 10 #validation Batch Size

h = 200 #height input
w= 200 #width input

valid_generator = dg.flow_from_directory(directory=vd_dir,
                                         batch_size = bsv,
                                         classes = classes,
                                         target_size= (h,w),
                                         class_mode = 'categorical',
                                         color_mode = 'rgb',
                                         seed = seed,
                                         shuffle = False)

train_gen = augment_generator.flow_from_directory(directory=trainingDir,
                                                   target_size =(h,w),
                                                   batch_size=bs,
                                                   classes = classes,
                                                   class_mode = 'categorical',
                                                   shuffle=True,
                                                   seed = seed,
                                                   color_mode = 'rgb'
                                                   )

# Creation of the datasets for training and validation

trainingDataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                                 output_types = (tf.float32,tf.float32),
                                                 output_shapes = ([None,h,w,3],[None,20]))

validationDataset = tf.data.Dataset.from_generator(lambda: valid_generator,
                                                  output_types = (tf.float32,tf.float32),
                                                  output_shapes =([None,h,w,3],[None,20]))

trainingDataset.repeat()
validationDataset.repeat()

Function for the creation of the CSV file

In [ ]:
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


## Building the CNN structure

In [ ]:
#Loss, Metrics, Optimizer and others

loss = tf.keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.7)
regularizer = tf.keras.regularizers.l2(0.01)

# Feature Extractor

fe = tf.keras.Sequential()
fe.add(tf.keras.layers.Conv2D(filters=64,input_shape=(h,w,3),kernel_size=(3,3),padding='same',activation=tf.keras.activations.relu))
fe.add(tf.keras.layers.MaxPool2D(pool_size=(4,4)))
fe.add(tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),padding='same',activation=tf.keras.activations.relu))
fe.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
fe.add(tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),padding='same',activation=tf.keras.activations.relu))
fe.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
fe.add(tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding='same',activation=tf.keras.activations.relu))
fe.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Fully Connected

fc = tf.keras.Sequential()
fc.add(tf.keras.layers.Flatten())
fc.add(tf.keras.layers.Dense(700,activation=tf.keras.activations.relu))
fc.add(tf.keras.layers.Dropout(0.3))
fc.add(tf.keras.layers.Dense(600,activation=tf.keras.activations.relu))
fc.add(tf.keras.layers.Dropout(0.3))
fc.add(tf.keras.layers.Dense(20,activation = tf.keras.activations.softmax))

#Complete Model

model = tf.keras.Sequential([fe,fc])
model.compile(optimizer= optimizer, loss = loss, metrics = metrics)
model.summary()
fe.summary()
fc.summary()

#Function for the Learning rate scheduler used in the fit method

def scheduler(epoch):
    if epoch < 20:
        return 0.04
    elif epoch >= 20 and epoch <= 30:
        return 0.01
    else:
        return 0.01 * 1.0/(1.0+ (epoch*0.07))

#Callbacks

tb_callback = tf.keras.callbacks.TensorBoard(log_dir = tb_dir,histogram_freq=1)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler,1)
es_callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',patience=3)

#Fitting for the model

model.fit(x = trainingDataset,
         epochs=50, verbose=2,
         steps_per_epoch = len(train_gen),
         validation_data = validationDataset,
         validation_steps = len(valid_generator),
         callbacks = [tb_callback,lr_scheduler, es_callback]
         )

#Save the structure of the model in a json file

m = model.to_json(indent=3)
with open(os.path.join(sv_dir,'json_model_'+now+'.json'), 'w') as out:
    out.write(m)

#Save the model in the h5 format

model.save(os.path.join(sv_dir,'CNN_model'+now+'.h5'))


In [ ]:
#Build the results on the model

results = {}
print('Test set predictions...')
image_filenames = os.walk(datasetDir+'/test')
for(root,dirs,images) in image_filenames:
    for image_name in images:
        img = Image.open(datasetDir+'/test/'+image_name).resize((h,w)).convert('RGB')
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, 0)
        img_array = tf.cast(img_array,tf.float32)/255
        res = model.predict(x = img_array)
        prediction = np.argmax(res)
        results[image_name] = prediction
print('Work done.')
print('Writing results...')
create_csv(results,csv_dir)
print('Results Written.')

ValidationCreation.delete_validation()